In [ ]:
import sys, os
from os.path import join
from os import path
import tomopy
import matplotlib.pyplot as plt
from getpass import getuser
import scipy
import ipywidgets
import numpy as np
import h5py

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
from interactive import cimshow

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget


## Load Data

In [ ]:
PROPOSAL = 11018117
USER = getuser()

# beamtime folder on maxwell
BASEFOLDER = "/data/beamtimes/PETRA/2308_MaxP04/%s/" % PROPOSAL
subfolder = "XPCS"
sample_name = "XPCS_Aug2023"

# processed folder on maxwell
#folder_general = sup.create_folder(join(BASEFOLDER, "processed"))
folder_general = join("/data/export/cklose/2308_PETRA_Skyrmion_CCI","Analysis","FTH_CDI")

In [ ]:
def load_fth(im_id):
    """
    Load fth dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_fth_%s.hdf5" % (im_id, USER),
    )
    print("Loading: %s"%fname)
    
    with h5py.File(fname, "r") as f:
        data = {}
        data["im_id"] = f["im_id"][()]
        data["roi"] = f["roi"][()]
        data["prop_dist"] = f["prop_dist"][()]
        data["phase"] = f["phase"][()]
        data["reco"] = f["recon"][()]
        data["s_rotx"] = f["experimental_setup"]["s_rotx"][()]
        data["s_rotz"] = f["experimental_setup"]["s_rotz"][()]
    return data

def load_cdi(im_id):
    """
    Load cdi dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_cdi_%s.hdf5" % (im_id, USER),
    )

    print("Loading: %s"%fname)

    with h5py.File(fname, "r") as f:
        data = {}
        data["im_id"] = f["im_id"][()]
        data["roi"] = f["roi"][()]
        data["prop_dist"] = f["prop_dist"][()]
        data["phase"] = f["phase"][()]
        data["supportmask"] = f["supportmask"][()]
        data["reco"] = f["reco"][()]
        data["reco_pc"] = f["reco_pc"][()]
        data["s_rotx"] = f["experimental_setup"]["s_rotx"][()]
        data["s_rotz"] = f["experimental_setup"]["s_rotz"][()]
    return data

In [ ]:
# Define scan id
im_id_set = [293,294]

print("Image Set:  %s" % im_id_set)

In [ ]:
proj, s_rotx, s_rotz = [], [], []

for im_id in im_id_set: 
    data = load_fth(im_id)
    #data = load_cdi(im_id)

    proj.append(data["reco"])
    s_rotx.append(data["s_rotx"])
    s_rotz.append(data["s_rotz"])

proj = np.stack(proj)
theta = np.array(s_rotx) # choose the relevant angle, either x or z

# Plot projection
fig, ax = cimshow(np.abs(proj),cmap='gray')
ax.set_title("Projections")

## Align Data

In [ ]:
proj, sx, sc, cov = tomopy.align_seq(proj, theta, center=rot_center, iters=10, algorithm="gridrec") # could take a lot of time, maybe reduce iters, no gpu acceleration
rot_center = tomopy.find_center(proj, theta) # find rotation center


## Reconstruct Data 

In [ ]:
extra_options = {'MinConstraint': 0.000, 'MaxConstraint':1 } # are there any known limitations which values the voxels can have? set here min and max for each voxel
options = {   'proj_type': 'cuda',    'method': 'SIRT_CUDA',    'num_iter': 500,    'extra_options': extra_options} # define reconstruction algorithm
recon = tomopy.recon(tomopy.minus_log(proj), #negativ log is needed if absorption contrast is used -> sum over absorption coefficient of each voxel, if phase is used this is not necessary
                     theta,
                     center=rot_center, 
                     algorithm=tomopy.astra,
                     options=options) #returns 3D array containing the reconstruction

## Plot Data

In [ ]:
%matplotlib widget
cimshow(recon) #x-z Plane
cimshow(np.rot90(recon)) #x-y Plane